# Import Testing Data

In [1]:
from Class.data import Data

CLASS_NAMES = ['MALADE', 'SAIN']
FOLDER = "epochs_10"

data_test = Data("test")
# data_train = Data("train")

Test Shape: (624, 172, 364, 1)


# Import Pretrained Models

In [3]:
from tensorflow.keras.models import load_model
from IPython.display import clear_output
import os

modelList = []
for fold in os.listdir('./saved_models/'):
    m = load_model('./saved_models/' + fold)
    modelList.append([m, fold])
    print("Imported: {}/{}".format(len(modelList), len(os.listdir('./saved_models/'))))
    clear_output(wait=True)
print("Import OK")

Import OK


# Compare Models

In [4]:
max_value = 0
i = 1
max_index = 0
max_name = ""
for model in modelList:
    print("{}/{} - {}".format(i, len(modelList), model[1]), end=" - ")
    loss, acc = model[0].evaluate(data_test.x_test, data_test.y_test, verbose=2)
    if (acc > max_value):
        max_value = acc
        max_name = model[1]
        max_index = i - 1
    i += 1

    # print(model[1] + ' - Accuracy: {:5.2f}%'.format(100 * acc))
print("Max accuracy: {:5.2f}%".format(max_value * 100))
print("Best model: {} at index: {}".format(max_name, max_index))

1/4 - 10_adam_32_90 - 

ValueError: in user code:

    /home/lucien/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1233 test_function  *
        return step_function(self, iterator)
    /home/lucien/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/lucien/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/lucien/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/lucien/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/lucien/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine

# MISC

In [4]:
# from Class.data import Data

# data_train = Data("train")

# max_value = 0
# i = 1
# max_name = ""
# for model in modelList:
#     print("{} - {}/{}".format(model[1], i, len(modelList)))
#     loss, acc = model[0].evaluate(data_train.x_train, data_train.y_train, verbose=2)
#     if (acc > max_value):
#         max_value = acc
#         max_name = model[1]
#     i += 1

#     # print(model[1] + ' - Accuracy: {:5.2f}%'.format(100 * acc))
# print("Max accuracy: {:5.2f}%".format(max_value * 100))
# print("Best model: {}".format(max_name))

Train Shape: (5216, 172, 364, 1)
5_avg_elu - 1/24
163/163 - 22s - loss: 0.1251 - accuracy: 0.9546
min_avg_relu - 2/24
163/163 - 20s - loss: 0.0198 - accuracy: 0.9937
5_max_relu - 3/24
163/163 - 19s - loss: 0.0515 - accuracy: 0.9780
5_avg_relu - 4/24
163/163 - 20s - loss: 0.0514 - accuracy: 0.9791
10_max_relu - 5/24
163/163 - 17s - loss: 0.0025 - accuracy: 0.9996
15_avg_elu - 6/24
163/163 - 24s - loss: 0.0889 - accuracy: 0.9670
20_avg_relu - 7/24
163/163 - 20s - loss: 0.0098 - accuracy: 0.9962
5_max_elu - 8/24
163/163 - 19s - loss: 0.0791 - accuracy: 0.9732
15_avg_relu - 9/24
163/163 - 19s - loss: 0.0051 - accuracy: 0.9985
50_avg_elu - 10/24
163/163 - 22s - loss: 0.0012 - accuracy: 0.9998
min_max_elu - 11/24
163/163 - 18s - loss: 0.0148 - accuracy: 0.9964
50_max_elu - 12/24
163/163 - 18s - loss: 0.5716 - accuracy: 0.7429
50_max_relu - 13/24
163/163 - 17s - loss: 1.3210e-06 - accuracy: 1.0000


In [5]:
# loss, acc = model.evaluate(data_train.x_train, data_train.y_train, verbose=2)

# print('Restored model, train accuracy: {:5.2f}%'.format(100 * acc))
# print(model.predict(data_train.x_train).shape)

In [19]:
import matplotlib.pyplot as plt
from random import randint
from tensorflow import nn
import numpy as np

error = 0
modelList[max_index][0].evaluate(data_test.x_test, data_test.y_test, verbose=2)

for i in range(data_test.x_test.shape[0]):
    # image_index = randint(0, x_test.shape[0] -1)
    image_index = i

    pred = modelList[max_index][0].predict(data_test.x_test[image_index].reshape(1, data_test.x_test[image_index].shape[0], data_test.x_test[image_index].shape[1], 1))
    score = nn.softmax(pred[0])
    value = np.argmax(score)
    res = CLASS_NAMES[value]

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(res, 100 * np.max(score))
    )
    
    if not int(data_test.y_test[image_index]) == int(value):
        error += 1

print("Total errors: " + str(error))
print("%: " + str(((error / data_test.x_test.shape[0]) * 100)))

ADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 72.86 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
This image most likely belongs to MALADE with a 73.11 percent confidence.
T